# 测试stdout输出

你的程序中有个方法会输出到标准输出中（ `sys.stdout` ）。

也就是说它会将文本打印到屏幕上面。 

你想写个测试来证明它，给定一个输入，相应的输出能正常显示出来。

**解决方案**

使用 `unittest.mock` 模块中的 `patch()` 函数， 使用起来非常简单，可以为单个测试模拟 `sys.stdout` 然后回滚， 并且不产生大量的临时变量或在测试用例直接暴露状态变量。

作为一个例子，我们在 mymodule 模块中定义如下一个函数：

In [1]:
%%file mymodule.py

def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

Writing mymodule.py


默认情况下内置的 `print` 函数会将输出发送到 `sys.stdout` 。

为了测试输出真的在那里，你可以使用一个替身对象来模拟它，然后使用断言来确认结果。

使用 `unittest.mock` 模块的 `patch()` 方法可以很方便的在测试运行的上下文中替换对象， 并且当测试完成时候自动返回它们的原有状态。

下面是对 `mymodule` 模块的测试代码：

In [5]:
%%file __init__.py
# for import

Writing __init__.py


In [17]:
from io import StringIO
from unittest import TestCase
from unittest.mock import patch
import mymodule

class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)
        with patch('sys.stdout', new=StringIO()) as fake_out:
            mymodule.urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)
            self.assertEqual(fake_out.getvalue(), 'expected_url')


In [18]:
t = TestURLPrint()
t.test_url_gets_to_stdout()

AssertionError: 'http://www.example.com\n' != 'expected_url'
- http://www.example.com
+ expected_url

`urlprint()` 函数接受三个参数，测试方法开始会先设置每一个参数的值。 

`expected_url` 变量被设置成包含期望的输出的字符串。

`unittest.mock.patch()` 函数被用作一个上下文管理器，使用 `StringIO` 对象来代替 `sys.stdout` . `fake_out` 变量是在该进程中被创建的模拟对象。

在 `with` 语句中使用它可以执行各种检查。

当 `with` 语句结束时， `patch` 会将所有东西恢复到测试开始前的状态

# 在单元测试中给对象打补丁

你写的单元测试中需要给指定的对象打补丁， 用来断言它们在测试中的期望行为（比如，断言被调用时的参数个数，访问指定的属性等）

**解决方案**

`unittest.mock.patch()` 函数可被用来解决这个问题。 

`patch()` 还可被用作一个装饰器、上下文管理器或单独使用，尽管并不常见。 

例如，下面是一个将它当做装饰器使用的例子：

```py
from unittest.mock import patch
import example

@patch('example.func')
def test1(x, mock_func):
    example.func(x)       # Uses patched example.func
    mock_func.assert_called_with(x)
```

它还可以被当做一个上下文管理器：

```py
with patch('example.func') as mock_func:
    example.func(x)      # Uses patched example.func
    mock_func.assert_called_with(x)
```    
    
最后，你还可以手动的使用它打补丁：

```py
p = patch('example.func')
mock_func = p.start()
example.func(x)
mock_func.assert_called_with(x)
p.stop()
```

如果可能的话，你能够叠加装饰器和上下文管理器来给多个对象打补丁。例如：

```py
@patch('example.func1')
@patch('example.func2')
@patch('example.func3')
def test1(mock1, mock2, mock3):
    ...

def test2():
    with patch('example.patch1') as mock1, \
         patch('example.patch2') as mock2, \
         patch('example.patch3') as mock3:
    ...
```


`patch()` 接受一个已存在对象的全路径名，将其替换为一个新的值。

原来的值会在装饰器函数或上下文管理器完成后自动恢复回来。 默认情况下，所有值会被 
`MagicMock` 实例替代

In [26]:
x = 42
with patch('__main__.x'):
     print(x)

<MagicMock name='x' id='2680791932488'>


不过，你可以通过给 `patch()` 提供第二个参数来将值替换成任何你想要的：

In [27]:
x = 42
with patch('__main__.x', 'patched_value'):
     print(x)

patched_value


被用来作为替换值的 `MagicMock` 实例能够模拟可调用对象和实例。 

他们记录对象的使用信息并允许你执行断言检查，例如：

In [28]:
from unittest.mock import MagicMock
m = MagicMock(return_value = 10)
m(1, 2, debug=True) # 10
m.assert_called_with(1, 2, debug=True)
m.assert_called_with(1, 2)

10

AssertionError: Expected call: mock(1, 2)
Actual call: mock(1, 2, debug=True)

In [36]:
m.upper.return_value = 'HELLO'
m.upper('hello') # 'HELLO'

assert m.upper.called # True

m.split.return_value = ['hello', 'world']
m.split('hello world') # ['hello', 'world']

m.split.assert_called_with('hello world') # Expected call ; Actual call

m['blah']

m.__getitem__.called # True
m.__getitem__.assert_called_with('blah') # Expected call ; Actual call

'HELLO'

['hello', 'world']

<MagicMock name='mock.__getitem__()' id='2680792450328'>

True

# 在单元测试中测试异常情况

对于异常的测试可使用 `assertRaises()` 方法。 例如，如果你想测试某个函数抛出了 `ValueError` 异常，像下面这样写：

In [43]:
import unittest

# A simple function to illustrate
def parse_int(s):
    return int(s)

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaises(ValueError, parse_int, 'N/A')

In [45]:
t = TestConversion()
t.test_bad_int()

如果你想测试异常的具体值，需要用到另外一种方法：

In [48]:
import errno
import unittest

class TestIO(unittest.TestCase):
    def test_file_not_found(self):
        try:
            f = open('/file/not/found')
        except IOError as e:
            self.assertEqual(e.errno, errno.ENOENT)

        else:
            self.fail('IOError not raised')

In [49]:
t = TestIO()
t.test_file_not_found()

## 讨论

`assertRaises()` 方法为测试异常存在性提供了一个简便方法。 

一个常见的陷阱是手动去进行异常检测

In [50]:

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        try:
            r = parse_int('N/A')
        except ValueError as e:
            self.assertEqual(type(e), ValueError)

这种方法的问题在于它很容易遗漏其他情况，比如没有任何异常抛出的时候。 那么你还得需要增加另外的检测过程，如下面这样：

In [ ]:
class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        try:
            r = parse_int('N/A')
        except ValueError as e:
            self.assertEqual(type(e), ValueError)
        else:
            self.fail('ValueError not raised')

`assertRaises()` 方法会处理所有细节，因此你应该使用它。

`assertRaises()` 的一个缺点是它测不了异常具体的值是多少。

为了测试异常值，可以使用 `assertRaisesRegex()` 方法， 它可同时测试异常的存在以及通过正则式匹配异常的字符串表示。例如：

```py
class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaisesRegex(
            ValueError, 
            'invalid literal .*',
            parse_int, 
            'N/A'
        )
```        
        
`assertRaises()` 和 `assertRaisesRegex()` 还有一个容易忽略的地方就是它们还能被当做上下文管理器使用：

```py
class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        with self.assertRaisesRegex(
            ValueError, 
            'invalid literal .*'
            ):
            
            r = parse_int('N/A')
```

但你的测试涉及到多个执行步骤的时候这种方法就很有用了。

# 将测试输出用日志记录到文件中